In [1]:
#Import necessary packages
import math
import numpy as np
np.set_printoptions(suppress=True,
                    formatter={'all': lambda x: '%5.3f' % x})
import pandas as pd
from scipy.optimize import brute, fmin, minimize
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

#Install import_ipynb so we can read dependenies as jupyter notebooks
!pip install ipynb

#Import dependencies
from ipynb.fs.full.Lewis_Integration_option_valuation import BCC_call_value
from ipynb.fs.full.CIR_calibration import CIR_calibration , r_list
from ipynb.fs.full.CIR_zcb_valuation import B

In [2]:
#Import CSV file with option data as a panda dataframe
raw = pd.read_csv('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/option_data.csv')

#Convert variables Maturity and Date to pandas datetime objects
raw['Maturity']= pd.to_datetime(raw['Maturity'], format='%Y-%m-%d') 
raw['Date']= pd.to_datetime(raw['Date'], format='%Y-%m-%d')

In [3]:
#Create a copy so we do not have to reload the data set if we start over
data = raw.copy()
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Maturity  159 non-null    datetime64[ns]
 1   Strike    159 non-null    float64       
 2   Call      159 non-null    float64       
 3   Date      159 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2)
memory usage: 5.1 KB


,Maturity,Strike,Call,Date
0,2020-10-29,205.0,21.500,2020-10-26
1,2020-10-29,210.0,16.375,2020-10-26
2,2020-10-29,215.0,11.500,2020-10-26
3,2020-10-29,220.0,6.950,2020-10-26
4,2020-10-29,225.0,3.050,2020-10-26
...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26
155,2021-12-17,270.0,5.250,2020-10-26
156,2021-12-17,290.0,2.450,2020-10-26
157,2021-12-17,310.0,1.100,2020-10-26


In [4]:
#Create a variable, T, that shows how much, as a fraction of a year, that is left until maturity
data['diff'] = data['Maturity'] - data['Date']                     #number of days left to maturity
data['T']=(data['diff'].astype('timedelta64[D]').astype(int))/365  

#Drop unnecessary varibles
data.drop(['diff'], axis = 1, inplace=True)
options = data
options

,Maturity,Strike,Call,Date,T
0,2020-10-29,205.0,21.500,2020-10-26,0.008219
1,2020-10-29,210.0,16.375,2020-10-26,0.008219
2,2020-10-29,215.0,11.500,2020-10-26,0.008219
3,2020-10-29,220.0,6.950,2020-10-26,0.008219
4,2020-10-29,225.0,3.050,2020-10-26,0.008219
...,...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26,1.142466
155,2021-12-17,270.0,5.250,2020-10-26,1.142466
156,2021-12-17,290.0,2.450,2020-10-26,1.142466
157,2021-12-17,310.0,1.100,2020-10-26,1.142466


In [5]:
#Initial short rate (Eonia 3.12.2020)
r0 = r_list[0]  

#Calibrate Short Rate Model to get calibrated estimates of CIR parameters used for zcb valuation
kappa_r, theta_r, sigma_r = CIR_calibration()

r = []
for row, option in options.iterrows():
    B0T = B([kappa_r, theta_r, sigma_r, r0, option['T']])
    r.append(-math.log(B0T) / option['T'])
options['r'] = r
options

Optimization terminated successfully.
         Current function value: 0.000020
         Iterations: 201
         Function evaluations: 382


,Maturity,Strike,Call,Date,T,r
0,2020-10-29,205.0,21.500,2020-10-26,0.008219,-0.004668
1,2020-10-29,210.0,16.375,2020-10-26,0.008219,-0.004668
2,2020-10-29,215.0,11.500,2020-10-26,0.008219,-0.004668
3,2020-10-29,220.0,6.950,2020-10-26,0.008219,-0.004668
4,2020-10-29,225.0,3.050,2020-10-26,0.008219,-0.004668
...,...,...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26,1.142466,0.000189
155,2021-12-17,270.0,5.250,2020-10-26,1.142466,0.000189
156,2021-12-17,290.0,2.450,2020-10-26,1.142466,0.000189
157,2021-12-17,310.0,1.100,2020-10-26,1.142466,0.000189


In [6]:
##################################################################################################################
################################################ Preliminaries ###################################################

#Define Stock price 
S0 = 226

#Slice the dataframe
#Default option selects all options in the csv file

#Choose only options that are within 2% of ATM 
#tol = 0.02  # percent ITM/OTM options
#options = [(np.abs(options['Strike'] - S0) / S0) < tol] 

#Choose specific strike prices 
# options = options[options['Strike'].isin([180, 200, 220, 240, 260])]

#Parameters from H93 & jump calibrations 
kappa_v, theta_v, sigma_v, rho, v0 = np.load('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/opt_sv.npy')
jump = np.load('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/opt_jump.npy')
lamb, mu, delta = jump[1:4] #Merton's calibration of sigma is not needed 
p0 = [kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta]

##################################################################################################################
######################################## Loss Function for the calibration #######################################

i = 0
min_MSE = 5000.0
def BCC_error_function(p0):
    ''' Error function for parameter calibration in BCC97 model via
    Lewis (2001) Fourier approach.

    Parameters
    ==========
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial, instantaneous variance
    lamb: float
        jump intensity
    mu: float
        expected jump size
    delta: float
        standard deviation of jump

    Returns
    =======
    MSE: float
        mean squared error
    '''
    global i, min_MSE
    kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta = p0 
    if kappa_v < 0.0 or theta_v < 0.005 or sigma_v < 0.0 or \
        rho < -1.0 or rho > 1.0 or v0 < 0.0 or lamb < 0.0 or \
        mu < -.6 or mu > 0.0 or delta < 0.0:
       return 5000.0
    if 2 * kappa_v * theta_v < sigma_v ** 2:
       return 5000.0

    se = []
    for row, option in options.iterrows():
        model_value = BCC_call_value(S0, option['Strike'], option['T'],
                            option['r'], kappa_v, theta_v, sigma_v, rho, v0,
                            lamb, mu, delta)
        se.append((model_value - option['Call']) ** 2)
    MSE = sum(se) / len(se)
    min_MSE = min(min_MSE, MSE)
    if i % 25 == 0:
        print('%4d |' % i, np.array(p0), '| %7.3f | %7.3f' % (MSE, min_MSE))
    i += 1
    return MSE

##################################################################################################################
############## Calibration of BCC (1997) parameters kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta ##########

def BCC_calibration_full():
    ''' Calibrates complete BCC97 model to market quotes. '''
    # local, convex minimization for all parameters
    opt = fmin(BCC_error_function, p0,  
                 xtol=0.001, ftol=0.001,
                 maxiter=450, maxfun=650)
    np.save('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/opt_sv_jump.npy', np.array(opt))
    return opt

##################################################################################################################
################### Calculation of BCC (1997) option values with stochastic interest rates  ######################

def BCC_calculate_model_values(p0):
    ''' Calculates all model values given parameter vector p0. '''
    kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta = p0  
    values = []
    for row, option in options.iterrows():
        model_value = BCC_call_value(S0, option['Strike'], option['T'],
                            option['r'], kappa_v, theta_v, sigma_v, rho, v0,
                            lamb, mu, delta)
        values.append(model_value)
    return np.array(values)

##################################################################################################################
####################################################### Run calibration ##########################################

if __name__ == '__main__':
    
    #Run and print the calibration
    opt = BCC_calibration_full()
    print('')
    print("Calibrated parameter vector: ", opt)

    #Create pandas column for Heston (1993) call options values based on calibrated data
    options['Model'] = BCC_calculate_model_values(opt)

   0 | [1.378393 0.028020 0.277406 -0.585100 0.070098 0.390309 -0.248649 0.000014] |   1.861 |   1.861
  25 | [1.549288 0.030680 0.269321 -0.623894 0.051989 0.392326 -0.246915 0.000014] |   0.918 |   0.918
  50 | [1.697457 0.024367 0.265068 -0.690285 0.062548 0.384280 -0.200292 0.000013] |   0.539 |   0.539
  75 | [1.753459 0.022888 0.263902 -0.718574 0.063091 0.376485 -0.173795 0.000013] |   0.435 |   0.388
 100 | [1.955506 0.017764 0.246369 -0.794753 0.082747 0.318617 -0.074775 0.000014] |   0.353 |   0.351
 125 | [1.923663 0.017156 0.255033 -0.786109 0.081375 0.323215 -0.085246 0.000014] |   0.349 |   0.348
 150 | [1.910602 0.017375 0.255002 -0.784531 0.081621 0.322793 -0.087815 0.000014] |   0.347 |   0.347
 175 | [1.893906 0.017426 0.256332 -0.781136 0.081365 0.322720 -0.092195 0.000014] |   0.347 |   0.346
 200 | [1.892487 0.017408 0.256611 -0.785283 0.081453 0.321139 -0.091813 0.000014] |   0.346 |   0.346
 225 | [1.885802 0.017699 0.257866 -0.796271 0.080995 0.321544 -0.092878 

In [7]:
#Drop max.rows option so we can see full dataframe to decide where implied vol gives NaN values
pd.set_option('display.max_rows', None)
options

,Maturity,Strike,Call,Date,T,r,Model
0,2020-10-29,205.0,21.500,2020-10-26,0.008219,-0.004668,20.996185
1,2020-10-29,210.0,16.375,2020-10-26,0.008219,-0.004668,16.008781
2,2020-10-29,215.0,11.500,2020-10-26,0.008219,-0.004668,11.081645
3,2020-10-29,220.0,6.950,2020-10-26,0.008219,-0.004668,6.481795
4,2020-10-29,225.0,3.050,2020-10-26,0.008219,-0.004668,2.873561
5,2020-10-29,230.0,0.850,2020-10-26,0.008219,-0.004668,0.850866
6,2020-10-29,235.0,0.360,2020-10-26,0.008219,-0.004668,0.149623
7,2020-11-06,205.0,21.625,2020-10-26,0.030137,-0.004556,21.129589
8,2020-11-06,210.0,17.000,2020-10-26,0.030137,-0.004556,16.394066
9,2020-11-06,215.0,12.500,2020-10-26,0.030137,-0.004556,11.977413
